In [3]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14

import seaborn as sns
import cv2
import pandas as pd
import numpy as np

import glob

import numpy as np
from PIL import Image, ImageDraw

from sklearn.preprocessing import LabelEncoder

import logging
# logging.basicConfig(filename='example.log',level=logging.DEBUG)

In [4]:
CLASSES_CSV = glob.glob('../input/train_simplified/*.csv')
CLASSES = [x.split('/')[-1][:-4] for x in CLASSES_CSV]

# 读取单个csv文件
def read_df(path, nrows):
    print('Reading...', path)
    if isinstance(nrows, int):
        return pd.read_csv(path, nrows=nrows, parse_dates=['timestamp'])
    else:
        return pd.read_csv(path, parse_dates=['timestamp'])

# 读取多个csv文件
def contcat_df(paths, nrows):
    dfs = []
    for path in paths:
        dfs.append(read_df(path, nrows))
    return pd.concat(dfs, axis=0, ignore_index=True)

df = contcat_df(CLASSES_CSV, 500)
df = df.reindex(np.random.permutation(df.index))

('Reading...', '../input/train_simplified/eye.csv')
('Reading...', '../input/train_simplified/castle.csv')
('Reading...', '../input/train_simplified/pizza.csv')
('Reading...', '../input/train_simplified/umbrella.csv')
('Reading...', '../input/train_simplified/bat.csv')
('Reading...', '../input/train_simplified/hot tub.csv')
('Reading...', '../input/train_simplified/diving board.csv')
('Reading...', '../input/train_simplified/wine bottle.csv')
('Reading...', '../input/train_simplified/butterfly.csv')
('Reading...', '../input/train_simplified/bread.csv')
('Reading...', '../input/train_simplified/television.csv')
('Reading...', '../input/train_simplified/peas.csv')
('Reading...', '../input/train_simplified/binoculars.csv')
('Reading...', '../input/train_simplified/basket.csv')
('Reading...', '../input/train_simplified/steak.csv')
('Reading...', '../input/train_simplified/eyeglasses.csv')
('Reading...', '../input/train_simplified/backpack.csv')
('Reading...', '../input/train_simplified/ted

('Reading...', '../input/train_simplified/octopus.csv')
('Reading...', '../input/train_simplified/diamond.csv')
('Reading...', '../input/train_simplified/wine glass.csv')
('Reading...', '../input/train_simplified/crab.csv')
('Reading...', '../input/train_simplified/blackberry.csv')
('Reading...', '../input/train_simplified/submarine.csv')
('Reading...', '../input/train_simplified/bee.csv')
('Reading...', '../input/train_simplified/The Eiffel Tower.csv')
('Reading...', '../input/train_simplified/finger.csv')
('Reading...', '../input/train_simplified/belt.csv')
('Reading...', '../input/train_simplified/jail.csv')
('Reading...', '../input/train_simplified/bracelet.csv')
('Reading...', '../input/train_simplified/megaphone.csv')
('Reading...', '../input/train_simplified/saw.csv')
('Reading...', '../input/train_simplified/microwave.csv')
('Reading...', '../input/train_simplified/pants.csv')
('Reading...', '../input/train_simplified/The Great Wall of China.csv')
('Reading...', '../input/train

('Reading...', '../input/train_simplified/lighthouse.csv')
('Reading...', '../input/train_simplified/t-shirt.csv')
('Reading...', '../input/train_simplified/ambulance.csv')
('Reading...', '../input/train_simplified/washing machine.csv')
('Reading...', '../input/train_simplified/police car.csv')
('Reading...', '../input/train_simplified/garden hose.csv')
('Reading...', '../input/train_simplified/owl.csv')
('Reading...', '../input/train_simplified/school bus.csv')
('Reading...', '../input/train_simplified/cow.csv')
('Reading...', '../input/train_simplified/triangle.csv')
('Reading...', '../input/train_simplified/toilet.csv')
('Reading...', '../input/train_simplified/monkey.csv')
('Reading...', '../input/train_simplified/wristwatch.csv')
('Reading...', '../input/train_simplified/floor lamp.csv')
('Reading...', '../input/train_simplified/bandage.csv')
('Reading...', '../input/train_simplified/harp.csv')
('Reading...', '../input/train_simplified/flashlight.csv')
('Reading...', '../input/tra

In [5]:
lbl = LabelEncoder().fit(df['word'])
df['word'] = lbl.transform(df['word'])

In [6]:
df_train = pd.read_pickle(os.path.join('./data', 'train_50000' + '.pkl'))
df_val = pd.read_pickle(os.path.join('./data', 'val_all' + '.pkl'))

In [7]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    BASE_SIZE = 299
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(eval(raw_strokes)):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i] + 22, stroke[1][i]  + 22),
                         (stroke[0][i + 1] + 22, stroke[1][i + 1] + 22), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

class QRDataset(Dataset):
    def __init__(self, img_drawing, img_label, img_size, transform=None):
        self.img_drawing = img_drawing
        self.img_label = img_label
        self.img_size = img_size
        self.transform = transform

    def __getitem__(self, index):
        img = np.zeros((self.img_size, self.img_size, 3))
        img[:, :, 0] = draw_cv2(self.img_drawing[index], self.img_size)
        img[:, :, 1] = img[:, :, 0]
        img[:, :, 2] = img[:, :, 0]
        img = Image.fromarray(np.uint8(img))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.from_numpy(np.array([self.img_label[index]]))
        return img, label

    def __len__(self):
        return len(self.img_drawing)

In [22]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

model = models.resnet50(True)
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc = nn.Linear(2048, 340)


model = nn.DataParallel(model).cuda()
model.load_state_dict(torch.load('./resnet50_256_0_28116(81.3451_93.1845).pt'))

optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2, 3, 5, 7, 8], gamma=0.1)

In [12]:
train_loader = torch.utils.data.DataLoader(
    QRDataset(df_train['drawing'].values, df_train['word'].values, 256,
                     transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        transforms.RandomAffine(5, scale=[0.85, 1.05]),
                        transforms.ToTensor(),
        ])
    ),
    batch_size=1500, shuffle=True, num_workers=10,
)

val_loader = torch.utils.data.DataLoader(
    QRDataset(df_val['drawing'].values, df_val['word'].values, 256,
                     transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        # transforms.RandomAffine(5, scale=[0.85, 1.05]),
                        transforms.ToTensor(),
        ])
    ),
    batch_size=1500, shuffle=True, num_workers=10,
)

import torch.nn.functional as F
loss_fn = nn.CrossEntropyLoss()

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
for epoch in range(10):
    scheduler.step()
    for i, data in enumerate(train_loader):
        x, y = data
        x = Variable(x).cuda(0)
        y = Variable(y.view(-1)).cuda(0)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            acc1, acc3 = accuracy(output, y, topk=(1, 3))
            logstr = 'Epoch {0}/{1}: \tloss {2}, ACC {3:.4f}/{4:.4f}'.format(epoch, i, loss.item(), 
                                                                  acc1.item(), acc3.item())
            logging.info(logstr)
        if i % 1000 == 0:
            torch.save(model.state_dict(), 'resnet18_{0}.pt'.format(epoch))

In [13]:
acc1_, acc2_, acc3_ = [], [], []
with torch.no_grad():
    model = model.eval()
    for data in val_loader:
        images, labels = data
        images = Variable(images).cuda(async=True)
        labels = Variable(labels.view(-1)).cuda()
        
        outputs = model(images)
        acc1, acc2, acc3 = accuracy(outputs, labels, topk=(1, 2, 3))
        
        acc1_.append(acc1.item())
        acc2_.append(acc2.item())
        acc3_.append(acc3.item())
        print acc1.item(), acc2.item(), acc3.item()
        
print np.mean(acc1_), np.mean(acc2_), np.mean(acc3_)
# 83.19421590016988 94.2012027809006

82.2666702271 91.1333389282 94.5333404541
81.9333343506 89.6666717529 92.6000061035
81.9333343506 90.2000045776 92.6666717529
82.8666687012 91.8666687012 94.6000061035
82.9333343506 91.2000045776 93.4000015259
80.8000030518 90.8666687012 93.1333389282
82.7333374023 91.8000030518 94.1333389282
81.8000030518 90.2666702271 92.5333404541
82.1333389282 90.6666717529 93.6000061035
82.1333389282 90.8666687012 93.0666732788
81.6666717529 91.4666748047 93.7333374023
82.6666717529 91.4666748047 93.8666687012
84.5333404541 92.6666717529 95.0666732788
83.0000076294 90.7333374023 93.93334198
82.8000030518 91.2000045776 93.5333404541
85.0000076294 92.0000076294 93.7333374023
81.2000045776 90.6000061035 93.2666702271
80.6666717529 90.0000076294 93.0666732788
82.5333404541 91.6000061035 94.1333389282
83.4666748047 91.5333404541 94.2000045776
81.1333389282 90.7333374023 93.8000030518
82.8000030518 91.4000015259 94.5333404541
84.6666717529 92.3333358765 94.6000061035
82.6666717529 92.0000076294 94.66667

83.5333404541 91.6666717529 93.6666717529
81.6000061035 90.9333343506 94.6000061035
83.1333389282 91.2000045776 93.0666732788
82.6666717529 90.8666687012 93.2000045776
82.8000030518 90.5333404541 93.0666732788
84.0666732788 92.5333404541 94.6000061035
82.2666702271 90.6666717529 94.0000076294
82.4666671753 91.1333389282 93.7333374023
80.7333374023 90.3333358765 92.7333374023
83.0000076294 92.0000076294 95.0000076294
82.5333404541 91.0666732788 93.8000030518
83.6000061035 92.2000045776 94.2000045776
83.0666732788 91.6666717529 94.4000015259
82.2000045776 91.4000015259 93.7333374023
83.8666687012 92.6000061035 94.7333374023
81.8666687012 91.6000061035 93.93334198
81.5333404541 89.8000030518 92.5333404541
83.3333358765 91.8000030518 93.5333404541
82.2000045776 89.9333343506 92.4000015259
82.8000030518 91.4000015259 93.8666687012
83.3333358765 90.1333389282 92.2000045776
82.0666732788 91.3333358765 94.1333389282
82.4666671753 89.9333343506 92.8666687012
82.0000076294 90.8666687012 93.53334

In [23]:
submit = pd.read_csv('../input/sample_submission.csv')
submit_df = pd.read_csv('../input/test_simplified.csv')

In [24]:
test_loader = torch.utils.data.DataLoader(
    QRDataset(submit_df['drawing'].values, np.zeros(submit_df.shape[0]), 256,
                     transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        # transforms.RandomAffine(5, scale=[0.85, 1.05]),
                        transforms.ToTensor(),
        ])
    ),
    batch_size=1500, shuffle=False, num_workers=10,
)

In [25]:
pred_tta = []
for tti in range(5):
    pred  = []
    with torch.no_grad():
        for t, (x, y) in enumerate(test_loader):
            x_var = Variable(x).cuda(async=True)
            y_var = Variable(y).cuda()
            scores = model(x_var)
            pred.append(scores.data.cpu().numpy())
    pred = np.concatenate(pred, 0)
    
    print(tti)
pred_tta.append(pred)

0
1
2
3
4


In [26]:
pred = np.mean(pred_tta, axis=0)

In [27]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pred_label = [lbl.inverse_transform(x.argsort()[-3:][::-1]) for x in pred]

In [28]:
pred_label = np.vstack(pred_label)

In [29]:
submit['top1'] = pred_label[:, 0]
submit['top2'] = pred_label[:, 1]
submit['top3'] = pred_label[:, 2]

submit['top1'] = submit['top1'].apply(lambda x: x.replace(' ', '_'))
submit['top2'] = submit['top2'].apply(lambda x: x.replace(' ', '_'))
submit['top3'] = submit['top3'].apply(lambda x: x.replace(' ', '_'))

In [30]:
submit['word'] = submit['top1'] + ' ' + submit['top2'] + ' ' + submit['top3']
submit[['key_id', 'word']].to_csv('./1116_82.270_93.630_tta5.csv', index=None)

In [36]:
pred_label

[array(['radio', 'stereo', 'binoculars'], dtype=object),
 array(['hockey puck', 'bottlecap', 'pool'], dtype=object),
 array(['The Great Wall of China', 'castle', 'fence'], dtype=object),
 array(['mountain', 'elbow', 'necklace'], dtype=object),
 array(['campfire', 'fireplace', 'fire hydrant'], dtype=object),
 array(['fence', 'jail', 'spreadsheet'], dtype=object),
 array(['wine glass', 'hourglass', 'shovel'], dtype=object),
 array(['submarine', 'dolphin', 'lobster'], dtype=object),
 array(['bracelet', 'hand', 'arm'], dtype=object),
 array(['hourglass', 'vase', 'wine glass'], dtype=object),
 array(['octopus', 'octagon', 'snowflake'], dtype=object),
 array(['stove', 'oven', 'dishwasher'], dtype=object),
 array(['camel', 'roller coaster', 'squiggle'], dtype=object),
 array(['stereo', 'skull', 'traffic light'], dtype=object),
 array(['harp', 'cello', 'violin'], dtype=object),
 array(['shorts', 'underwear', 'pants'], dtype=object),
 array(['sailboat', 'cruise ship', 'speedboat'], dtype=object

In [20]:
pred_label

array([[237, 284,  32],
       [147,  39, 229],
       [  1,  65, 113],
       ...,
       [238,  76, 228],
       [ 42, 228,  50],
       [ 70, 310,  27]])

In [1]:
import torch

In [2]:
torch.backends.cudnn.enabled

True